# Advanced Certification in AIML
## A Program by IIIT-H and TalentSprint

## Learning Objectives

At the end of the experiment, you will be able to:

* Understand the challenges of information retrieval on Wikipedia articles
* Weighted kNN algorithm for relevance feedback 
* Rocchio algorithm for relevance feedback 

In [ ]:
#@title Experiment Explanation Video
from IPython.display import HTML

HTML("""<video width="850" height="480" controls>
  <source src="https://cdn.talentsprint.com/talentsprint/archives/sc/aiml/aiml_batch_14/rocchios_algorithm.mp4" type="video/mp4">
</video>
""")

## Dataset

### History:
Wikipedia dataset is the most widely-used dataset for information retrieval. It is based on Wikipedia’s "featured articles", a continually updated article collection. There are totally 29 categories in "featured articles". Each article is split into several sections according to its section headings. This dataset is finally generated as a set of 2,866 image/text pairs.

As an important benchmark dataset for information retrieval, The wikipedia dataset has been widely used since being publicly available.

**Wikipedia featured articles:**

Featured articles are considered to be some of the best articles Wikipedia has to offer, as determined by Wikipedia's editors. They are used by editors as examples for writing other articles. Articles are reviewed as featured article candidates for accuracy, neutrality, completeness, and style, before being added to . There are 5,410 featured articles out of 5,754,701 articles on the English Wikipedia (~0.1% are featured).

### Description:

The dataset contains the following:
1. 2866 Wikipedia articles belonging to a total of 29 categories.
2. Each article is given a query ID based on it's category.
    



### Challenges:

The following challenges are associated with query based retrieval of documents:

1.   Incomplete query or semantically wrong query
2.   Huge number of documents returns as a result to the query

## Domain Information



Information Retrieval (IR) systems allow users to access large amounts of data. A user submitting a request to an IR system will receive a number of results relating to the request. The results may include images, pieces of text,
web pages, segments of video or speech samples. 

But, the users may not have a well-developed idea of what information they are searching for, they may not be able to express entirely what they want into a suitable query and they may not have a good idea of what information is available for retrieval. 

To solve the above problem, the system after presenting the user with an initial set of documents a feedback mechanism could identify those documents that had useful information. This is the Relevance Feedback (RF). The system then can use this information quantitatively - retrieving more documents like the relevant documents - and qualitatively - retrieving only those documents similar to the relevant documents. Therefore RF is a cycle.

The image below helps us understand the Relevance Feedback cycle:

<img src = "
https://cdn.talentsprint.com/aiml/Experiment_related_data/IMAGES/1.1.png" width = "700" height = "500">

## AI / ML Technique

### Relevance Feedback for Information Retrieval

In this experiment, dataset is queried to get relevant documents. The significant part of any Information retrieval system is to make it responsive to user interaction. Hence our system needs to incorporate the feedback and refine the results using the feedback on the fly.

Search engines like Google, Bing and other websites like Youtube are use - cases which personalize the content for  better user experience.

In this experiment we implement two algorithms to achieve adaptive retrieval based on user input.

Below steps are specific to this experiment. In the real-word, based on the usecase, few or all of these steps can be applied.

1. #### Weighted kNN 

     The steps are:

    - Random list of wikipedia documents (varied category) are displayed to the User. 
    - The user based on his interest clicks on any document belonging to a particular category. 
    - We implement a weighted kNN to get a ranking of the relevant documents based on the user input.
    - The user then again clicks a document of the relevant documents.
    - We update the weight vector to get a new ranking which ensures better recall than before


2. #### Rocchio Algorithm

     The steps are:
      
    - Random list of documents are displayed.
    - The user clicks on a particular document and also decides the relevant and non relevant documents of the displayed ones.
    - The initial document clicked is our Query $q$ (initial centroid for relevant documents). <br> Relevant Documents are $D_{r}$, Non relevant ones are $D_{nr}$.
    - We use the following formula to converge to a new centroid for the relevant documents.<br> This ensures better precision with every iteration

$$q_{t+1} = a . q_{t} + b. (\frac{1}{|D_{r}|} \sum_{d_{j} \in D_{r}} d_{j}) - c. (\frac{1}{|D_{nr}|} \sum_{d_{k} \in D_{nr}} d_{k})$$

- Display the nearest neighbours of the new centroid $q_{t+1}$, which would be more relevant



In [ ]:
! wget https://cdn.talentsprint.com/aiml/Experiment_related_data/week12/Exp1/wikipedia_dataset.tar.gz
! tar xvf  wikipedia_dataset.tar.gz

### Importing required packages

In [ ]:
from glob import glob
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
import random
from bs4 import BeautifulSoup as Soup
from sklearn.neighbors import NearestNeighbors
import pandas as pd

from IPython.display import clear_output
from IPython.display import display, HTML
import matplotlib.pyplot as plt

In [ ]:
f = open('wikipedia_dataset/categories_originalids.list')
cat = f.readlines()
print('The following ' + str(len(cat)) + ' categories', cat)

### Parsing the XML data files 

In [ ]:
def parseLog(file, docs, qid):
    # Read the file
    handler = open(file).read()
    # To the Soup pass the data to be parsed and the name of a parser 'lxml' as parameters
    soup = Soup(handler,'lxml')
    # Finding the text
    txt = soup.findAll('text')[0].text       
    for message in soup.findAll('document'):
        # message.attrs attribute, which returns a dictionary of key-value pairs
        msg_attrs = dict(message.attrs)
        qid.append(int(msg_attrs['cat']))
        docs.append(msg_attrs['name'] + ' ' + txt)

lis = glob('wikipedia_dataset/texts/*')
docs = []
qid = []
for i in range(len(lis)):
    parseLog(lis[i], docs, qid)

In [ ]:
print('Sample document:', qid[100], docs[100][:100] + '...')

### Construct feature vector from documents

In [ ]:
stoplist = open('wikipedia_dataset/stopwords.txt').read().splitlines()

In [ ]:
# TF-IDF Vectorizer
vectorizer = TfidfVectorizer(stop_words=stoplist, min_df=0.001)
matrix = vectorizer.fit_transform(docs)
doc_vectors = matrix.todense()
doc_vectors = np.array(doc_vectors)
vocab = vectorizer.vocabulary_

In [ ]:
print('Length of the vocabulary:', len(vocab))
print('A sample ' +str(len(vocab)) +' dimensional document vector:', doc_vectors[2][:100])

In [ ]:
print(doc_vectors.shape, len(qid))

### 1. Weighted KNN 
- Initially display random 10 documents and get user click
- Run the KNN Algorithm and choose 100 nearest neighbours
- Define a weight matrix to get the ranking of the 100 relevant vectors based on similarity

Given $m$ - dimensional query $q$ and document $d_{i}$, the weighted similarity is calculated as follows <br> 
$q = (q_{1}, q_{2}, \cdots, q_{m})$ <br>
$d_{i} = (d_{1}, d_{2}, \cdots, c_{m})$ <br>
$$Similarity = w^T d'_{i}$$ where ,
$$d'_{i} = \frac{q * d_{i}}{\parallel q \parallel \parallel d_{i} \parallel}  $$

In [ ]:
# Fit KNN Model
KNN = NearestNeighbors(100, metric = 'cosine')
KNN.fit(doc_vectors)
neighbours = KNN.kneighbors(doc_vectors,return_distance=False)
print(neighbours.shape)

In [ ]:
# Number of articles to be displayed 
num = 10

In [ ]:
def to_display(doc_ids, init= False):
    if init == True:
        ix = random.sample(range(0, len(doc_vectors)), num)
    else:
        ix = doc_ids
    
    df = pd.DataFrame(columns=['index', 'Article','category'])
    df['index'] = ix
    df['Article'] = [docs[i][:100] for i in ix]
    df['category'] = [cat[qid[i]-1][:-1] for i in ix] 
    clear_output()
    display(HTML(df.to_html()))
    return ix

index = to_display([],init= True)
res = []
ctr = 0

# Initialize the weight vector to ones
weights = np.ones((doc_vectors.shape[1],1))
while (True):  
    doc_id = input("Select the index of the article you would like to read or type 'stop' if you want to end the search")
    if doc_id == 'stop':
        break    
    doc_id = int(doc_id) 

    query_doc = doc_vectors[doc_id].reshape(1, doc_vectors.shape[1])
    relevant_docs_ix = neighbours[doc_id]
    relevant_docs = doc_vectors[relevant_docs_ix]

    # Calculate Similarity
    new_relevance = (query_doc[0] * relevant_docs) / (np.linalg.norm(relevant_docs) * np.linalg.norm(query_doc))
    similarity = np.matmul(new_relevance , weights)

    # Return the number of most ranked doc ids
    ranked_docs_ix = relevant_docs_ix[np.argsort(similarity[:,0])]
    index = to_display(ranked_docs_ix[:num])
    
    weights = weights + doc_vectors[doc_id].reshape(doc_vectors.shape[1], 1)
    weights = weights/np.linalg.norm(weights)
    ctr += 1
    res_ = input("Give the precision count\a number between 1-10 about how satisfied are you with the results")
    res.append(res_)

print('Enjoy your Article')
plt.plot(res)
plt.xlabel("number of searches")
plt.ylabel("Score given by you")

### 2. Rocchio Algorithm
- Initially random articles are displayed
- Select an index value based on your choice
- Select all the indices of relevant documents
- Update the centroid of relevant vector as shown in the formula above
- Display the nearest neighbors of the relevant vector

- $alpha$ - how close to relevant 
- $beta$ - how far from non relevant

Ideally we would like to have high $alpha$ and low $beta$

In [ ]:
alpha = 1.0
beta = 0.5
index = to_display([], init= True)
while(True):
    doc_id = input("Select the index of the article you would like to read or type 'stop' if you want to end the search")
    if doc_id == 'stop':
        break
    print("Enter the indices, comma separated for relevant documents")
    R = [int(x) for x in input().split(',')]
    NR = [i for i in index if i not in R]
    doc_id = int(doc_id)
    query_doc = doc_vectors[doc_id].reshape(1, doc_vectors.shape[1])
    R_docs = doc_vectors[R]
    NR_docs = doc_vectors[NR]

    q_new = query_doc + (alpha * np.sum(R_docs, 0)/len(R)) - (beta * np.sum(NR_docs, 0)/len(NR))
    print(q_new.shape)
    retrieved = KNN.kneighbors(q_new, return_distance=False)
    index = to_display(retrieved[0][:num])
    
print('Enjoy your article')